In [45]:
import pandas as pd
import numpy as np

In [122]:
trials = pd.read_json('./dynomite_trials.json')
# mods table
mods = pd.read_json('./dynomite_mods.json')
# split into hp/torque trials
hp_trials = trials.loc[trials['data_type_id'] == 1]
torque_trials = trials.loc[trials['data_type_id'] == 2]
trials

,display_info,name,displayName,owner_id,car_id,run_id,data_type_id,mods,run_data
0,{'username': '11secondFWD'},11secondFWD HP 9 1,11secondFWD HP (17g),9,9,1,1,"{'4': 'td04', '5': '17g'}","[{'x': 2510, 'y': 53}, {'x': 2921, 'y': 107}, ..."
1,{'username': '11secondFWD'},11secondFWD TQ 9 1,11secondFWD TQ (17g),9,9,1,2,"{'4': 'td04', '5': '17g'}","[{'x': 2514, 'y': 118}, {'x': 2619, 'y': 143},..."
2,{'username': '2root4u'},2root4u HP 18 1,2root4u HP (gt368),18,18,1,1,"{'1': '25', '4': 'td04', '5': 'gt368'}","[{'x': 3564, 'y': 210}, {'x': 3942, 'y': 262},..."
3,{'username': '3sx'},3sx HP 6 1,3sx HP (GT35),6,6,1,1,"{'4': 'td05', '5': 'GT35', '8': '100'}","[{'x': 4200, 'y': 200}, {'x': 4800, 'y': 370},..."
4,{'username': '97vr4'},97vr4 HP 15 1,97vr4 HP (19t),15,15,1,1,"{'1': '25', '4': 'td04', '5': '19t'}","[{'x': 3457, 'y': 201}, {'x': 3686, 'y': 244},..."
...,...,...,...,...,...,...,...,...,...
113,{'username': 'vr4usmc203'},vr4usmc203 TQ 52 2,vr4usmc203 TQ (13t),48,52,2,2,"{'2': 'race', '4': 'td04', '5': '13t', '11': '...","[{'x': 2758, 'y': 282}, {'x': 3032, 'y': 334},..."
114,{'username': 'widowmaker'},widowmaker HP 44 1,widowmaker HP (evo3 16g),41,44,1,1,"{'1': '19', '2': 'pump', '3': '3.5', '4': 'td0...","[{'x': 3677, 'y': 207}, {'x': 3923, 'y': 247},..."
115,{'username': 'Wraith'},Wraith HP 22 1,Wraith HP (dr650r),22,22,1,1,"{'1': '19', '4': 'td04', '5': 'dr650r'}","[{'x': 3824, 'y': 217}, {'x': 3891, 'y': 229},..."
116,{'username': 'wsrtt'},wsrtt HP 27 1,wsrtt HP (9b),27,27,1,1,"{'1': '21', '4': 'td04', '5': '9b', '7': '50/50'}","[{'x': 3023, 'y': 203}, {'x': 3165, 'y': 222},..."


In [123]:
def q_summary(numbers):
    outliers_text = "The following are outliers: {}"
    # to calculate outliers
    quartiles = [np.quantile(numbers, 1/4), np.quantile(numbers, 2/4), np.quantile(numbers, 3/4)]
    IQR = 1.5*(quartiles[2]-quartiles[0])
    outliers = [n for n in numbers if (n < quartiles[0]-IQR or n > quartiles[2]+IQR)]
    withoutOutliers = [n for n in numbers if n not in outliers]
    # now print
    ret = {
        'min':np.min(withoutOutliers),
        'q1':quartiles[0],
        'q2':quartiles[1],
        'q3':quartiles[2],
        'max':np.max(withoutOutliers),
        'outliers':outliers if outliers else None
    }
    return ret

In [135]:
# Run data has a different number of points for each run. 'outliers' is not a good question to ask without refinement
# because these are continous functions.
# We will do an outlier/min/max summary on the integral of each run, because a run with higher horsepower 
# would have a bigger integral, which is what we are interested in.
# Runs have different start and end points (RPM), so we will take a median of start/end to use as the integration bounds.

# for horsepower
hp_start_rpms = [run[0]['x'] for run in hp_trials['run_data']] # only interested in 'x' part, i.e. rpm
hp_end_rpms = [run[-1]['x'] for run in hp_trials['run_data']]
hp_start_summary = q_summary(hp_start_rpms)
hp_end_summary = q_summary(hp_end_rpms)
print('HP start summary: {}'.format(hp_start_summary))
print('HP end summary: {}'.format(hp_end_summary))
hp_start = hp_start_summary['q2']
hp_end = hp_end_summary['q2']
print('Integration bounds (median q2): ({}, {})'.format(hp_start, hp_end), end='\n\n')
# for torque
torque_start_rpms = [run[0]['x'] for run in torque_trials['run_data']]
torque_end_rpms = [run[-1]['x'] for run in torque_trials['run_data']]
torque_start_summary = q_summary(torque_start_rpms)
torque_end_summary = q_summary(torque_end_rpms)
print('Torque start summary: {}'.format(torque_start_summary))
print('Torque end summary: {}'.format(torque_end_summary))
torque_start = torque_start_summary['q2']
torque_end = torque_end_summary['q2']
print('Integration bounds (median q2): ({}, {})'.format(torque_start, torque_end))

# filter function to 'chop off' the parts of a run that are outside of the bounds
def filter_run(run):
    return list(filter(run[0]['x'] >= hp_start and run[-1]['x'] <= hp_end), run)

# now do the integrals - use numpy discrete integral (trapezoidal rule)
hp_integrals = []
filtered_hp_trials = [filter_run(run) for run in hp_trials['run_data']]
print(hp_trials['run_data'][0])
print(filtered_hp_trials[0])
for trial in hp_trials['run_data']:
    xs = [point['x'] for point in trial]
    ys = [point['y'] for point in trial]
    integral = np.trapz(ys, x=xs)
    hp_integrals.append(integral)

HP start summary: {'min': 1020, 'q1': 2510.0, 'q2': 3065.0, 'q3': 3603.0, 'max': 4672, 'outliers': None}
HP end summary: {'min': 5988, 'q1': 6639.0, 'q2': 6965.0, 'q3': 7158.0, 'max': 7544, 'outliers': [8500, 7950, 8011, 8463, 8468, 8072, 8015]}
Integration bounds (median q2): (3065.0, 6965.0)

Torque start summary: {'min': 1916, 'q1': 2400.0, 'q2': 2536.0, 'q3': 2882.0, 'max': 3300, 'outliers': [1012]}
Torque end summary: {'min': 6399, 'q1': 6758.0, 'q2': 6961.0, 'q3': 7100.0, 'max': 7563, 'outliers': [7950, 6103]}
Integration bounds (median q2): (2536.0, 6961.0)


TypeError: filter expected 2 arguments, got 1